# Silver Layer

- amount > 0 and not null
- valid currencies only
- valid statuses: ["AUTHORISED", "SETTLED", "REFUNDED", "CHARGEBACK", "DECLINED", "PENDING", "SUCCESS", "FAILED"]

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home"

spark = (
    SparkSession.builder
    .appName("PaymentsBronzeTest")
    .master("local[*]")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("Spark version:", spark.version)


In [ ]:
bronze_df = spark.read.parquet("../data/bronze/transactions_parquet/")

bronze_df.show(5)

In [ ]:
bronze_df.select("currency").distinct().show()

In [11]:
def validate_amount(df):
    """
    Validates the 'amount' column in a DataFrame.
    This function filters the DataFrame to include only rows where the 'amount' 
    is greater than 0 and is not null.
    Args:
        df (DataFrame): The input DataFrame containing a column named 'amount'.
    Returns:
        DataFrame: A filtered DataFrame containing only valid rows based on the 
        'amount' column criteria.
    """

    valid_df = df.where((F.col("amount") > 0) & F.col("amount").isNotNull())

    return valid_df

In [12]:
def validate_currency_types(df):
    """
    Validate the currency types in the given DataFrame.
    This function filters the DataFrame to include only rows where the 
    'currency' column contains one of the valid currency codes: 
    'USD', 'EUR', 'GBP', 'JPY', 'AUD', or 'CAD'.
    Args:
        df (DataFrame): The input DataFrame containing a 'currency' column.
    Returns:
        DataFrame: A DataFrame containing only the rows with valid currency types.
    """

    valid_df = df.where(F.col("currency").isin("USD", "EUR", "GBP", "JPY", "AUD", "CAD"))

    return valid_df

In [13]:
def validate_txn_statuses(df): 
    """
    Validate transaction statuses in a DataFrame.
    This function filters the input DataFrame to include only rows 
    where the 'status' column contains valid transaction statuses.
    Args:
        df (DataFrame): The input DataFrame containing transaction data.
    Returns:
        DataFrame: A DataFrame containing only the rows with valid statuses.
    """

    valid_df = df.where(F.col("status").isin(
        ["AUTHORISED", "SETTLED", "REFUNDED", "CHARGEBACK", "DECLINED", "PENDING", "SUCCESS", "FAILED"]
        ))

    return valid_df

In [15]:
def curate_status(df):
    """
    Map raw transaction statuses into curated categories 
    for downstream analytics, while preserving raw status.

    Args:
        df (DataFrame): Input DataFrame with a 'status' column.

    Returns:
        DataFrame: DataFrame with an additional 'status_curated' column.
    """
    mapping_expr = (
        F.when(F.col("status") == "AUTHORISED", "PENDING")
         .when(F.col("status") == "SETTLED", "SUCCESS")
         .when(F.col("status") == "REFUNDED", "REFUNDED")
         .when(F.col("status") == "CHARGEBACK", "FAILED")
         .when(F.col("status") == "DECLINED", "FAILED")
         .when(F.col("status") == "PENDING", "PENDING")
         .when(F.col("status") == "SUCCESS", "SUCCESS")
         .when(F.col("status") == "FAILED", "FAILED")
         .otherwise("UNKNOWN")
    )

    return df.withColumn("status_curated", mapping_expr)

In [ ]:
curate_status(bronze_df).show()